In [1]:
# 영화 리뷰 감성 분석 (감성평가)  (교재 p.416~)

In [2]:
from sklearn.datasets import load_files

In [3]:
reviews_train = load_files('data-files/aclImdb/train') #지정된 폴더의 하위폴더가 타겟 레이블

In [4]:
print(reviews_train.keys())
print(len(reviews_train['data']))
print(reviews_train['data'][0])
print(reviews_train['target_names'])
print(reviews_train['target'])

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])
25000
b"Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty."
['neg', 'pos']
[1 0 1 ... 0 0 0]


In [5]:
X_train, y_train = reviews_train['data'], reviews_train['target']

In [6]:
import numpy as np
import random

In [7]:
random.seed(0)
size = int(len(X_train) * 0.2)            # 25000건중 20%만 자르는 과정
# idxes = np.random.randint(0, len(X_train), size)  # 0부터 X_train갯수 까지중 size만큼 랜덤으로 뽑는다. 20% 데이터 샘플링 (복원추출)
idxes = random.sample(range(len(X_train)), size)  # 랜덤으로 추출 // 20% 데이터 샘플링(비복원추출)
print( len(idxes), idxes[:5], len(np.unique(idxes)))

5000 [12623, 24836, 13781, 1326, 8484] 5000


In [8]:
# X_train2, y_train2 = [X_train[i] for i in idxes],[y_train[i] for i in idxes]
X_train2, y_train2 = np.array(X_train)[idxes], np.array(y_train)[idxes]
X_train2.shape, y_train2.shape

((5000,), (5000,))

In [9]:
X_train2[1]   #데이터가 html로 되어 있음. <br /><br />
X_train2 = np.array([x.replace(b'<br />', b'') for x in X_train2])   #반복문 돌리며 진행(내포방식)     #자바스크립트 정규표현식 참조할것!
# 또는 아래방식으로도 진행 가능
# X_train22 = map(lambda x : x.replace(b'<br />', b''), X_train2)  
# X_train2 = np.array(list(X_train2))

X_train2[1]

b"Friz Freleng's 'Speedy Gonzalez' was the second cartoon to feature the title character after Robert McKimson's 'Cat-tails for Two'. In that cartoon, Speedy has been an ugly little creature with a big gold tooth but by his second appearance the famous design had already been adopted. Despite looking significantly more handsome, Speedy never developed into much of a character. A big hat, tremendous speed and a bad Mexican accent do not a classic character make and that's pretty much all Speedy ever had going for him. Nevertheless, the cocky little mouse proved enormously popular and went on to star in many shorts including some truly abysmal films from the studio's latter days. While these early Speedy shorts are better than those later atrocities in which he was frequently (rather oddly) paired up with Daffy Duck, they still leave much to be desired, relying on predictable gags usually based around a similar chase formula. In this self-titled episode, Speedy is recruited by some other

In [10]:
# 불균형성
np.unique(y_train2, return_counts=True)  # 값이 바뀌는 이유 => 랜덤값//  random.seed를 줘야 같은조건으로 동작된다.

(array([0, 1]), array([2505, 2495], dtype=int64))

In [11]:
reviews_test = load_files('data-files/aclImdb/test/')

In [12]:
X_test, y_test = reviews_test['data'], reviews_test['target']
size = int(len(X_test) * 0.2 )
idxes = random.sample(range(len(X_test)),size)
len(np.unique(idxes))

5000

In [13]:
X_test2 = np.array( [X_test[idx] for idx in idxes])
y_test2 = np.array( [y_test[idx] for idx in idxes])

In [14]:
np.unique(y_test2, return_counts=True)

(array([0, 1]), array([2544, 2456], dtype=int64))

In [15]:
X_test2 = np.array([ x.replace(b'<br />', b'') for x in X_test2])

In [16]:
X_test2[1]

b"What a great film! I never knew much about Buddy Holly, but was familiar with his lively and fun music. This is a wonderful biography of someone who helped change the music in the 1950's. Although I never cared for Gary Bussey, he was fabulous as Buddy Holly! I don't know how accurate the movie is, but assume at least for the most part it is accurate, which makes the movie all the more interesting. The music throughout the movie just adds the pizazz to this biography. I don't think I would change a thing in this film, it was all good! What a difference in the stars from the 50's to todays music stars. How can you compare someone like Buddy Holly to Justin Timberlake? or any of the other popular singers of this generation?"

 BOW example  (빈도 가중치)

In [17]:
test_words = ["The fool doth think he is wise",
              "but the wise man knows himself to be a fool"]

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
cv=CountVectorizer() # 문장 -> 단어사전 만들기 + 단어별 빈도수 계산
cv.fit(test_words)

CountVectorizer()

In [20]:
cv.vocabulary_      

{'the': 9,
 'fool': 3,
 'doth': 2,
 'think': 10,
 'he': 4,
 'is': 6,
 'wise': 12,
 'but': 1,
 'man': 8,
 'knows': 7,
 'himself': 5,
 'to': 11,
 'be': 0}

In [21]:
sorted(cv.vocabulary_, key=lambda key: cv.vocabulary_[key])

['be',
 'but',
 'doth',
 'fool',
 'he',
 'himself',
 'is',
 'knows',
 'man',
 'the',
 'think',
 'to',
 'wise']

In [22]:
import pandas as pd

# print( cv.transform(test_words) )
print(cv.transform(test_words).toarray())
pd.DataFrame(cv.transform(test_words).toarray(), 
             columns=sorted(cv.vocabulary_, key=lambda key: cv.vocabulary_[key]))

[[0 0 1 1 1 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 1 0 1 1 1 0 1 1]]


,be,but,doth,fool,he,himself,is,knows,man,the,think,to,wise
0,0,0,1,1,1,0,1,0,0,1,1,0,1
1,1,1,0,1,0,1,0,1,1,1,0,1,1


In [23]:
cv = CountVectorizer() #문자열을 숫자열로 변형
cv.fit(X_train2)
X_train3 = cv.transform(X_train2)

In [24]:
print(X_train3.toarray().shape)
print(X_train3.toarray()[:5])
# print(X_train3)
# print(cv.vocabulary_)
# cv.get_feature_names()   #정렬된 컬럼 이름들

(5000, 38559)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score #감성평가 분류

In [26]:
lr = LogisticRegression(solver='lbfgs',max_iter=10000, C=0.1)
lr.fit(X_train3, y_train2)
print(lr.score(X_train3, y_train2))
# print(lr.score(X_test3, y_test2))

0.9938


In [27]:
X_test3 = cv.transform(X_test2)

In [28]:
print(lr.score(X_test3, y_test2))

0.8602


In [29]:
print(average_precision_score(y_test2,lr.predict_proba(X_test3)[:,1]))
print(roc_auc_score(y_test2,lr.predict_proba(X_test3)[:,1]))

0.9188213820147895
0.928311713836478


In [30]:
cv = CountVectorizer(min_df=5) #5회 이상 발견된 데이터만 포함
cv.fit(X_train2)
print(len(cv.get_feature_names()))
X_train3=cv.transform(X_train2)
X_test3=cv.transform(X_test2)

11326


In [31]:
lr=LogisticRegression(max_iter=10000, C=0.1)
lr.fit(X_train3,y_train2)
print(lr.score(X_train3, y_train2))
print(lr.score(X_test3, y_test2))

0.9884
0.861


In [32]:
X_train2[0]

b'I saw this not too long ago, and I must say: This movie is terrible. I watch crappy movies for fun. Scarecreow is not fun. Scarecrow is stupid. You have an incredibly corny villain that enjoys screaming awful puns as he kills his victims(actually worse than the one contained in this sentence). He has his hard luck story that he uses to justify his killings. "Everyone picks on me. The only girl that thinks I\'m not trailer-trash likes one of the guys that pick on me. I want to kill everybody. Wah." OK, I\'m exaggerating. But the premise to this movie alone is enough to put it near the bottom of the list of crappy movies.Adding to what I just said, the kid\'s mom is promiscuous, he walks in on his mother and her current boyfriend getting it on, mom\'s boyfriend tells him to leave, kid refuses, insisting that he isn\'t going to leave his own house. Boyfriend chases kid into corn field. He kills kid right in front of mom, mom screams in terror, boyfriend is like, "OMG! I didn\'t mean to!

2) stop word

In [33]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
list(ENGLISH_STOP_WORDS)[:10]

['eight',
 'should',
 'though',
 'at',
 'made',
 'eg',
 'most',
 'rather',
 'will',
 'yourselves']

In [34]:
cv = CountVectorizer(min_df=5, stop_words=ENGLISH_STOP_WORDS)
# cv = CountVectorizer(min_df=5, stop_words="english")

cv.fit(X_train2)
X_train3 = cv.transform(X_train2)
X_test3 = cv.transform(X_test2)

In [35]:
lr = LogisticRegression(max_iter=1000, C=0.1)
lr.fit(X_train3,y_train2)
lr.score(X_train3,y_train2), lr.score(X_test3, y_test2)

(0.9852, 0.8574)

In [36]:
# 빈도 계산 방법 (p.429)

from sklearn.feature_extraction.text import TfidfVectorizer 

tv = TfidfVectorizer(min_df=5) # 단어의 상대빈도 계산(단어빈도/문서빈도)
tv.fit(X_train2)
X_train3=tv.transform(X_train2)
X_test3= tv.transform(X_test2)

In [41]:
lr = LogisticRegression(max_iter=1000, C=10)
lr.fit(X_train3, y_train2)
lr.score(X_train3,y_train2), lr.score(X_test3, y_test2)

(0.996, 0.8692)

토큰화, 어간추출,표제어추출 (p438)

In [4]:
# !pip install spacy # 표제어 사전 관리도구 설치
# !python -m spacy download en # 영어 표제어 사전 설치

In [1]:
import nltk
import spacy

In [2]:
stemmer = nltk.stem.PorterStemmer() # 어간추출기
en_nlp = spacy.load('en') # 영어 표제어 사전 가져오기

In [3]:
data = u"Our meeting today was worse than yesterday, I'm scared of meeting the clients tomorrow"

In [4]:
spacy_doc = en_nlp(data)
# print("분리된 단어 목록: " , [token for token in spacy_doc])   # 형태소 분리된 단어 목록 출력
print("표제어: " , [token.lemma_ for token in spacy_doc]) # 개별 단어의 표제어 출력
print("정규화 표현: " , [token.norm_ for token in spacy_doc])  # 개별 단어의 정규화 표현 출력
print("어간: " , [stemmer.stem(token.norm_) for token in spacy_doc])  #개별 단어의 어간 출력

표제어:  ['-PRON-', 'meeting', 'today', 'be', 'bad', 'than', 'yesterday', ',', '-PRON-', 'be', 'scared', 'of', 'meet', 'the', 'client', 'tomorrow']
정규화 표현:  ['our', 'meeting', 'today', 'was', 'worse', 'than', 'yesterday', ',', 'i', 'am', 'scared', 'of', 'meeting', 'the', 'clients', 'tomorrow']
어간:  ['our', 'meet', 'today', 'wa', 'wors', 'than', 'yesterday', ',', 'i', 'am', 'scare', 'of', 'meet', 'the', 'client', 'tomorrow']


In [5]:
# import re
# from sklearn.feature_extraction.text import CountVectorizer
# regexp = re.compile('(?u)\b\w\w+\b') #

# def custom_tokenizer(scr):
#     from spacy.tokens import Doc
#     doc = Doc(en_nlp.vocab, words=regexp.findall(scr))
#     return [token.lemma_ for token in doc]

In [6]:
from sklearn.datasets import load_files

In [7]:
reviews_train = load_files('data-files/aclImdb/train')

In [8]:
X_train, y_train = reviews_train['data'], reviews_train['target']

In [9]:
import numpy as np
import random

In [10]:
random.seed(0)
size = int(len(X_train) * 0.2)            # 25000건중 20%만 자르는 과정
# idxes = np.random.randint(0, len(X_train), size)  # 0부터 X_train갯수 까지중 size만큼 랜덤으로 뽑는다. 20% 데이터 샘플링 (복원추출)
idxes = random.sample(range(len(X_train)), size)  # 랜덤으로 추출 // 20% 데이터 샘플링(비복원추출)
print( len(idxes), idxes[:5], len(np.unique(idxes)))

5000 [12623, 24836, 13781, 1326, 8484] 5000


In [11]:
X_train2, y_train2 = np.array(X_train)[idxes], np.array(y_train)[idxes]
X_train2.shape, y_train2.shape

((5000,), (5000,))

In [12]:
X_train2[1]   #데이터가 html로 되어 있음. <br /><br />
X_train2 = np.array([x.replace(b'<br />', b'') for x in X_train2])   #반복문 돌리며 진행(내포방식)     #자바스크립트 정규표현식 참조할것!
# 또는 아래방식으로도 진행 가능
# X_train22 = map(lambda x : x.replace(b'<br />', b''), X_train2)  
# X_train2 = np.array(list(X_train2))

X_train2[1]

b"Friz Freleng's 'Speedy Gonzalez' was the second cartoon to feature the title character after Robert McKimson's 'Cat-tails for Two'. In that cartoon, Speedy has been an ugly little creature with a big gold tooth but by his second appearance the famous design had already been adopted. Despite looking significantly more handsome, Speedy never developed into much of a character. A big hat, tremendous speed and a bad Mexican accent do not a classic character make and that's pretty much all Speedy ever had going for him. Nevertheless, the cocky little mouse proved enormously popular and went on to star in many shorts including some truly abysmal films from the studio's latter days. While these early Speedy shorts are better than those later atrocities in which he was frequently (rather oddly) paired up with Daffy Duck, they still leave much to be desired, relying on predictable gags usually based around a similar chase formula. In this self-titled episode, Speedy is recruited by some other

In [13]:
reviews_test = load_files('data-files/aclImdb/test/')

In [14]:
X_test, y_test = reviews_test['data'], reviews_test['target']
size = int(len(X_test) * 0.2 )
idxes = random.sample(range(len(X_test)),size)
len(np.unique(idxes))

5000

In [15]:
X_test2 = np.array( [X_test[idx] for idx in idxes])
y_test2 = np.array( [y_test[idx] for idx in idxes])

In [16]:
X_test2 = np.array([ x.replace(b'<br />', b'') for x in X_test2])

In [17]:
X_test2[1]

b"What a great film! I never knew much about Buddy Holly, but was familiar with his lively and fun music. This is a wonderful biography of someone who helped change the music in the 1950's. Although I never cared for Gary Bussey, he was fabulous as Buddy Holly! I don't know how accurate the movie is, but assume at least for the most part it is accurate, which makes the movie all the more interesting. The music throughout the movie just adds the pizazz to this biography. I don't think I would change a thing in this film, it was all good! What a difference in the stars from the 50's to todays music stars. How can you compare someone like Buddy Holly to Justin Timberlake? or any of the other popular singers of this generation?"

In [26]:
import re
from sklearn.feature_extraction.text import CountVectorizer
regexp = re.compile('(?u)\\b\\w\\w+\\b') # (r'(?u)\b\w\w+\b')

def custom_tokenizer(scr):
    from spacy.tokens import Doc
    doc = Doc(en_nlp.vocab, words=regexp.findall(scr))
    return [token.lemma_ for token in doc]

In [27]:
cv = CountVectorizer(tokenizer=custom_tokenizer, min_df=5)
cv.fit(X_train2)
X_train3 = cv.transform(X_train2)
X_test3 = cv.transform(X_test2)

In [29]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000, C=0.1)
lr.fit(X_train3, y_train2)
print(lr.score(X_train3, y_train2), lr.score(X_test3, y_test2))

0.9784 0.8588
